In [1]:
spark

In [2]:
! rm -rf metastore_db/

In [3]:
#spark.stop()

In [4]:
spark = SparkSession \
    .builder \
    .appName("Prediction of Loan Payment 20 Feb 2020") \
    .master("local[*]") \
    .config("spark.executor.cores","3")\
    .getOrCreate();

# 1. Data Understanding

## Collecting the current data from the LendingClub's Website.

In [5]:
#! rm -rf LoanStats_web.csv

In [6]:
#! touch LoanStats_web.csv

In [7]:
! ls -l /root/notebook/data/LoanStats_web.csv

-rw-r--r-- 1 root root 1160243241 Nov 22 18:37 /root/notebook/data/LoanStats_web.csv


In [8]:
! head -3 /root/notebook/data/LoanStats_web.csv

"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","open_il_12m","open_il_24m","mths_since_rcnt_il","total_bal_il","il_util","open_rv_1

In [9]:
! wc -l /root/notebook/data/LoanStats_web.csv

1432493 /root/notebook/data/LoanStats_web.csv


In [10]:
raw_df = spark.read.format('csv').option('header','true').option('mode','DROPMALFORMED')\
.load('/root/notebook/data/LoanStats_web.csv')

In [11]:
raw201617_df = raw_df

In [12]:
raw201617_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

### Numbers of Fields [1]

In [13]:
len(raw201617_df.columns)

144

In [14]:
rawweb_df = raw201617_df.select('id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 #'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m')

### Numbers of Fields after attributes selections [1]

In [15]:
len(rawweb_df.columns)

73

In [16]:
rawweb_df.select(['issue_d']).distinct().show()

+--------+
| issue_d|
+--------+
|Oct-2016|
|Feb-2019|
|Mar-2018|
|Sep-2018|
|Jan-2016|
|Jul-2016|
|Aug-2016|
|Feb-2017|
|Nov-2018|
|Jul-2018|
|Apr-2018|
|Dec-2018|
|May-2016|
|Jan-2018|
|Aug-2018|
|Jan-2019|
|Apr-2016|
|Sep-2016|
|Oct-2018|
|Feb-2018|
+--------+
only showing top 20 rows



In [17]:
raw_df = rawweb_df

In [18]:
raw_df.count()

1432439

In [19]:
df_colfam5 = raw_df.select("loan_amnt","term","int_rate","installment","grade","emp_length",\
                           "home_ownership","annual_inc","verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs","earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d")

### Numbers of Fields after final attributes selections [1]

In [20]:
len(df_colfam5.columns)

22

In [21]:
! pip install pandas

You are using pip version 9.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [22]:
import pandas as pd
pd.DataFrame(df_colfam5.describe().take(6), columns=df_colfam5.describe().columns).transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
loan_amnt,1432439,15370.392945179516,9646.028077240424,1000,9975
term,1432439,None,None,36 months,60 months
int_rate,1432439,None,None,5.31%,30.99%
installment,1432439,456.6879090138958,281.71351826414445,100.01,999.98
grade,1432439,None,None,A,G
emp_length,1432439,None,None,1 year,n/a
home_ownership,1432439,None,None,ANY,RENT
annual_inc,1432439,81034.5941268703,134183.40313571738,0,99999.84
verification_status,1432439,None,None,Not Verified,Verified


df_colfam5.select(['last_credit_pull_d']).distinct().\
orderBy(['last_credit_pull_d'],ascending=1).show()

df_colfam5.select(['last_credit_pull_d']).\
filter(df_colfam5.last_credit_pull_d == '').count()

df_colfam5.select(['last_credit_pull_d']).\
filter(df_colfam5['last_credit_pull_d'].isNull()).count()

In [23]:
df_colfam5.groupby('home_ownership').count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|           OWN|170888|
|          RENT|556960|
|      MORTGAGE|701646|
|           ANY|  2940|
|          NONE|     5|
+--------------+------+



# 2. Data Preparation

Divide this process into 2 parts. The first is a business oriented preparation that turn many business rules to be programming's logics. Its result benefits many tasks related to the Business Intelligence and other descriptive analytics. The second one is a data science oriented preparation that turn many requirements of data science to be programming's logics. Its result benefits many tasks related to the Predictive analytics.

Asumption: Business oriented preparation: No missing values, ONLY month, Correct data types.

## Data Cleansing: Remove missing values

In [24]:
df_no_missing = df_colfam5.dropna(how='any')

## Check for a schema

In [25]:
df_no_missing.printSchema()

root
 |-- loan_amnt: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [26]:
df_no_missing_fitmem = df_no_missing.repartition(60)

In [27]:
df_no_missing_cached = df_no_missing_fitmem.cache()

In [28]:
df_no_missing_cached.registerTempTable("df")

## Number of data rows

In [29]:
df_no_missing_cached.count()

1429645

## Data Transformation: Remove Sign of Percent and Extract Month.

In [30]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

### Remove Sign of Percent

In [31]:
def f_removepercentsign(origin):
    return origin.rstrip('%')

In [32]:
removepercentsign = udf(lambda x: f_removepercentsign(x),StringType())

### Extract Month

In [33]:
def f_extractmonth(origin):
    return origin.split('-')[0]

In [34]:
extractmonth = udf(lambda x: f_extractmonth(x),StringType())

In [35]:
extractterm = udf(lambda x: x.replace('months',''),StringType())

In [36]:
#extractemp_length = udf(lambda x: x.replace('year',''),StringType())

In [37]:
from pyspark.sql.functions import col

In [38]:
dfWithCrunch = df_no_missing_cached.\
withColumn('revol_util',removepercentsign(col('revol_util')).cast(DoubleType())).\
withColumn('int_rate',removepercentsign(col('int_rate')).cast(DoubleType())).\
withColumn('earliest_cr_line',extractmonth(col('earliest_cr_line')).cast(StringType())).\
withColumn('last_credit_pull_d',extractmonth(col('last_credit_pull_d')).cast(StringType())).\
withColumn('dti',col('dti').cast(DoubleType())).\
withColumn('loan_amnt',col('loan_amnt').cast(DoubleType())).\
withColumn('revol_bal',col('revol_bal').cast(DoubleType())).\
withColumn('term',extractterm(col('term')).cast(DoubleType())).\
withColumn('installment',col('installment').cast(DoubleType())).\
withColumn('open_acc',col('open_acc').cast(DoubleType())).\
withColumn('total_acc',col('total_acc').cast(DoubleType())).\
withColumn('pub_rec',col('pub_rec').cast(DoubleType())).\
withColumn('annual_inc',col('annual_inc').cast(DoubleType()))

In [39]:
#dfWithCrunch.select(['last_credit_pull_d']).printSchema()

In [40]:
#dfWithCrunch.groupby('emp_length').count().show()

In [41]:
#dfWithCrunch.count()

In [42]:
dfWithCrunch.select(['loan_amnt','int_rate','dti','home_ownership','grade','term']).show()

+---------+--------+-----+--------------+-----+----+
|loan_amnt|int_rate|  dti|home_ownership|grade|term|
+---------+--------+-----+--------------+-----+----+
|  14975.0|   11.99|12.75|          RENT|    C|36.0|
|   5000.0|   11.99|17.04|          RENT|    C|36.0|
|  20000.0|    7.89|18.69|      MORTGAGE|    A|60.0|
|   9950.0|   20.99|13.36|          RENT|    E|36.0|
|  10000.0|   11.99|12.96|      MORTGAGE|    C|36.0|
|  11000.0|   17.27|13.89|          RENT|    D|36.0|
|  12000.0|   11.99|22.97|      MORTGAGE|    C|36.0|
|  11000.0|   14.46|30.56|          RENT|    C|36.0|
|   5000.0|   14.46|37.13|          RENT|    C|36.0|
|  21000.0|   12.99|18.27|      MORTGAGE|    C|36.0|
|  20000.0|   13.67| 9.23|      MORTGAGE|    C|36.0|
|  14000.0|   16.29|17.73|          RENT|    D|36.0|
|   6400.0|   18.99|16.59|      MORTGAGE|    D|36.0|
|  21000.0|   16.29|24.76|      MORTGAGE|    D|36.0|
|   3875.0|   12.99|22.52|      MORTGAGE|    C|36.0|
|   9000.0|   12.99|33.04|      MORTGAGE|    C

In [43]:
#dfWithCrunch.select(['loan_amnt','int_rate','dti','home_ownership','grade']).printSchema()

In [44]:
dfWithCrunch.select(['loan_amnt','annual_inc']).describe().show()

+-------+------------------+------------------+
|summary|         loan_amnt|        annual_inc|
+-------+------------------+------------------+
|  count|           1429645|           1429645|
|   mean|15366.699302973815| 81111.66344812173|
| stddev| 9641.851365996221|134222.54004920376|
|    min|            1000.0|               0.0|
|    max|           40000.0|             6.1E7|
+-------+------------------+------------------+



In [45]:
rawhive_df = dfWithCrunch.repartition(60).cache()
#rawhive_df.registerTempTable("crunched_data")

In [46]:
len(rawhive_df.columns)

22

In [47]:
rawhive_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [48]:
#rawhive_df.select(['loan_amnt','int_rate','dti','home_ownership','grade']).printSchema()

In [49]:
rawhive_df.count()

1429645

### Data Transformation: Normalization of "annual_inc" and "loan_amnt"

In [50]:
from pyspark.sql.functions import *

max_annual_inc = rawhive_df.select(max('annual_inc')).collect()[0][0]

min_annual_inc = rawhive_df.select(min('annual_inc')).collect()[0][0]

#sqlContext.udf.register("t_annual_inc", lambda x: ((x-min_annual_inc)/(max_annual_inc-min_annual_inc)))

In [51]:
def t_annual_inc(origin):
    return ((origin-min_annual_inc)/(max_annual_inc-min_annual_inc))

In [52]:
normalized_annual_inc = udf(lambda x: t_annual_inc(x),DoubleType())

In [53]:
max_loan_amnt = rawhive_df.select(max('loan_amnt')).collect()[0][0]

min_loan_amnt = rawhive_df.select(min('loan_amnt')).collect()[0][0]

#sqlContext.udf.register("t_loan_amnt", lambda x: ((x-min_loan_amnt)/(max_loan_amnt-min_loan_amnt)))

In [54]:
def t_loan_amnt(origin):
    return ((origin-min_loan_amnt)/(max_loan_amnt-min_loan_amnt))

In [55]:
normalized_loan_amnt = udf(lambda x: t_loan_amnt(x),DoubleType())

In [56]:
normalized_df = rawhive_df.withColumn('loan_amnt',normalized_loan_amnt(col('loan_amnt'))).\
withColumn('annual_inc',normalized_annual_inc(col('annual_inc')))

In [57]:
normalized_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



### Number of data rows that are only "Fully Paid" and "Charged Off"

In [58]:
from pyspark.sql.functions import col

In [59]:
normalized_df.select(col('loan_status')).groupBy('loan_status').count().show()

+------------------+------+
|       loan_status| count|
+------------------+------+
|        Fully Paid|582923|
|           Default|  1416|
|   In Grace Period|  6125|
|       Charged Off|152817|
|Late (31-120 days)| 19405|
|           Current|661982|
| Late (16-30 days)|  4977|
+------------------+------+



In [60]:
normalized_df.select(['loan_amnt','term','int_rate','installment','grade','annual_inc','revol_bal','loan_status']).\
orderBy(['loan_amnt','term','int_rate','installment','grade','annual_inc']).show()

+---------+----+--------+-----------+-----+--------------------+---------+-----------+
|loan_amnt|term|int_rate|installment|grade|          annual_inc|revol_bal|loan_status|
+---------+----+--------+-----------+-----+--------------------+---------+-----------+
|      0.0|36.0|    5.31|      30.12|    A|4.098360655737705E-4|   3215.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|5.081967213114754E-4|  25993.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|5.901639344262295E-4|   3311.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|6.065573770491804E-4|   1605.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|6.557377049180328E-4|   1329.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|6.557377049180328E-4|   1016.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|7.180327868852459E-4|    804.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|9.344262295081968E-4|    858.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|

In [61]:
normalized_df.select(['loan_amnt','annual_inc']).describe().show()

+-------+------------------+--------------------+
|summary|         loan_amnt|          annual_inc|
+-------+------------------+--------------------+
|  count|           1429645|             1429645|
|   mean|0.3683769052044867|0.001329699400788...|
| stddev|0.2472269581024673|0.002200369509003...|
|    min|               0.0|                 0.0|
|    max|               1.0|                 1.0|
+-------+------------------+--------------------+



In [62]:
normalized_filtered_df = normalized_df.filter(col('loan_status') == 'Fully Paid').\
union(normalized_df.filter(col('loan_status') == 'Charged Off'))

In [63]:
data = normalized_filtered_df.repartition(60).cache()

### Drop Null

In [64]:
data_no_missing_df = data.dropna(how='any')

In [65]:
data_no_missing_df.count()

735740

In [66]:
df_no_missing_cached.select(['loan_status']).distinct().show(100)

+------------------+
|       loan_status|
+------------------+
|        Fully Paid|
|           Default|
|   In Grace Period|
|       Charged Off|
|Late (31-120 days)|
|           Current|
| Late (16-30 days)|
+------------------+



In [67]:
data_no_missing_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



# 3. Data Modeling

In [68]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder,\
VectorIndexer, QuantileDiscretizer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

In [69]:
installmentDiscretizer = QuantileDiscretizer(numBuckets=5,inputCol='installment',\
                                             outputCol='installmentDiscreted')

In [70]:
labelIndexer = StringIndexer(inputCol='loan_status',outputCol='indexedLabel')

In [71]:
homeIndexer = StringIndexer(inputCol='home_ownership',outputCol='homeIndexed')
homeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='homeIndexed',outputCol='homeVec')

In [72]:
gradeIndexer = StringIndexer(inputCol='grade',outputCol='gradeIndexed')
gradeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='gradeIndexed',outputCol='gradeVec')

In [73]:
purposeIndexer = StringIndexer(inputCol='purpose',outputCol='purposeIndexed')
purposeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='purposeIndexed',\
                                     outputCol='purposeVec')

In [74]:
emp_lengthIndexer = StringIndexer(inputCol='emp_length',outputCol='emp_lengthIndexed')
emp_lengthOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='emp_lengthIndexed',\
                                     outputCol='emp_lengthVec')

In [75]:
featureAssembler = VectorAssembler(inputCols=[\
                                              'loan_amnt',\
                                              'int_rate',\
                                              #'revol_bal',\
                                              'annual_inc',\
                                              'emp_lengthVec',\
                                              'installmentDiscreted',\
                                              'dti',\
                                              'homeVec',\
                                              'gradeVec',\
                                              'open_acc',\
                                              'term'],\
                                   outputCol='features')

In [76]:
data_no_missing_df_fully_paid = data_no_missing_df.filter(col('loan_status') == 'Fully Paid').sample(True, 0.3, 42)

In [77]:
data_no_missing_df_charge_off = data_no_missing_df.filter(col('loan_status') == 'Charged Off')

In [78]:
final_data_no_missing_df = data_no_missing_df_fully_paid.union(data_no_missing_df_charge_off)

In [79]:
training , test = final_data_no_missing_df.filter(col('loan_amnt') > 0).randomSplit([0.8,0.2])

In [80]:
training.select('loan_amnt').describe().show()

+-------+-------------------+
|summary|          loan_amnt|
+-------+-------------------+
|  count|             261081|
|   mean| 0.3599745201386097|
| stddev|0.23792131931116164|
|    min|6.41025641025641E-4|
|    max|                1.0|
+-------+-------------------+



In [81]:
training.select(['revol_bal']).\
describe().show()

+-------+------------------+
|summary|         revol_bal|
+-------+------------------+
|  count|            261081|
|   mean|15827.285742738843|
| stddev|21259.651267874946|
|    min|               0.0|
|    max|         1113293.0|
+-------+------------------+



In [82]:
training.select(['loan_amnt','term','int_rate','installment','annual_inc']).\
describe().show()

+-------+-------------------+------------------+------------------+-----------------+--------------------+
|summary|          loan_amnt|              term|          int_rate|      installment|          annual_inc|
+-------+-------------------+------------------+------------------+-----------------+--------------------+
|  count|             261081|            261081|            261081|           261081|              261081|
|   mean|0.35997452013860975| 42.07195468073127|14.076170230694961|462.7690351270295|0.001275761027873...|
| stddev|0.23792131931116156|10.433537063068387| 5.410906563621437|284.8206310024222|0.001170442670087...|
|    min|6.41025641025641E-4|              36.0|              5.31|            32.29|                 0.0|
|    max|                1.0|              60.0|             30.99|          1715.42| 0.15693560655737704|
+-------+-------------------+------------------+------------------+-----------------+--------------------+



In [83]:
training

DataFrame[loan_amnt: double, term: double, int_rate: double, installment: double, grade: string, emp_length: string, home_ownership: string, annual_inc: double, verification_status: string, loan_status: string, purpose: string, addr_state: string, dti: double, delinq_2yrs: string, earliest_cr_line: string, inq_last_6mths: string, open_acc: double, pub_rec: double, revol_bal: double, revol_util: double, total_acc: double, last_credit_pull_d: string]

In [84]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

pipeline = Pipeline(stages=[gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    #home_ownershipIndexer,home_ownershipOneHotEncoder,\
                                    installmentDiscretizer,\
                                    labelIndexer,featureAssembler])

In [85]:
train_df_features = pipeline.fit(training).transform(training)

### MLP: กำหนด Pipeline เพื่อ Train Network

In [86]:
layers = [
    train_df_features.schema["features"].metadata["ml_attr"]["num_attrs"],
    20, 10, 2
]

In [87]:
layers

[30, 20, 10, 2]

In [88]:
clf = MultilayerPerceptronClassifier(labelCol='indexedLabel',featuresCol='features'\
                                     ,layers=layers)

In [89]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

pipeline = Pipeline(stages=[gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    #home_ownershipIndexer,home_ownershipOneHotEncoder,\
                                    installmentDiscretizer,\
                                    labelIndexer,featureAssembler,clf])

In [90]:
#getattr(clf,x1)

In [91]:
#getattr(clf,x2)

In [92]:
x1 = 'stepSize'
x2 = 'maxIter'
paramGrid = ParamGridBuilder() \
    .addGrid(getattr(clf,x1), [0.1, 0.2, 0.3, 0.4, 0.5]) \
    .addGrid(getattr(clf,x2),[5,10,15,20,25,30])\
    .build()
evaluator = MulticlassClassificationEvaluator\
(labelCol='indexedLabel', predictionCol='prediction', metricName='f1')

crossval = CrossValidator(estimator=pipeline,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=evaluator,
                                      numFolds=2)
cvModel = crossval.fit(training)

In [93]:
cvModel

CrossValidatorModel_4a73ab0c8dc3157dbb16

model1 = pipeline_dt.fit(training)

In [94]:
training.select(['loan_amnt','int_rate','dti','home_ownership','grade']).describe().show()

+-------+-------------------+------------------+------------------+--------------+------+
|summary|          loan_amnt|          int_rate|               dti|home_ownership| grade|
+-------+-------------------+------------------+------------------+--------------+------+
|  count|             261081|            261081|            261081|        261081|261081|
|   mean| 0.3599745201386098|14.076170230694963|  19.2482525729563|          null|  null|
| stddev|0.23792131931116156| 5.410906563621439|14.083055362056102|          null|  null|
|    min|6.41025641025641E-4|              5.31|               0.0|           ANY|     A|
|    max|                1.0|             30.99|             999.0|          RENT|     G|
+-------+-------------------+------------------+------------------+--------------+------+



In [95]:
training.select(['loan_amnt','int_rate','dti','home_ownership','grade']).show()

+--------------------+--------+-----+--------------+-----+
|           loan_amnt|int_rate|  dti|home_ownership|grade|
+--------------------+--------+-----+--------------+-----+
|0.002564102564102564|   16.99|16.56|          RENT|    D|
|0.005128205128205128|    6.72| 8.87|          RENT|    A|
|0.005128205128205128|    8.24|19.12|          RENT|    B|
|0.005128205128205128|    8.59|33.92|           OWN|    A|
|0.005128205128205128|    8.59|33.92|           OWN|    A|
|0.005128205128205128|    8.59|33.92|           OWN|    A|
|0.005128205128205128|    9.75| 8.29|      MORTGAGE|    B|
|0.005128205128205128|   11.44|12.72|          RENT|    B|
|0.005128205128205128|   11.47|16.97|      MORTGAGE|    B|
|0.005128205128205128|   13.49|  0.5|          RENT|    C|
|0.005128205128205128|   13.56|19.12|           OWN|    C|
|0.005128205128205128|   17.99|34.05|          RENT|    D|
|0.005128205128205128|   17.99|20.03|           OWN|    D|
|0.005128205128205128|   23.99| 9.74|      MORTGAGE|    

# 4. Evaluation of Models

In [96]:
raw_result = cvModel.transform(test)

In [97]:
spark_labeling = udf(lambda x: "Charged Off" if x == 1.0 else "Fully Paid")

In [98]:
result = raw_result.withColumn('prediction_label',spark_labeling(col('prediction')))

In [99]:
test.count()

65041

In [100]:
training.count()

261081

In [101]:
training.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|139044|
|Charged Off|122037|
+-----------+------+



In [102]:
(0.25*349664)

87416.0

In [103]:
training.filter(training.loan_status == 'Fully Paid').count()

139044

In [104]:
training.filter(training.loan_status == 'Charged Off').count()

122037

In [105]:
result.select('loan_status','indexedLabel','prediction','prediction_label')\
.filter(result.loan_status != 'Fully Paid').show(10)

+-----------+------------+----------+----------------+
|loan_status|indexedLabel|prediction|prediction_label|
+-----------+------------+----------+----------------+
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged Off|         1.0|       0.0|      Fully Paid|
|Charged Off|         1.0|       0.0|      Fully Paid|
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged Off|         1.0|       0.0|      Fully Paid|
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged Off|         1.0|       0.0|      Fully Paid|
|Charged Off|         1.0|       0.0|      Fully Paid|
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged Off|         1.0|       1.0|     Charged Off|
+-----------+------------+----------+----------------+
only showing top 10 rows



In [106]:
result.select('loan_status','indexedLabel','prediction')\
.filter(result.loan_status != 'Fully Paid').count()

30266

In [107]:
result.select('loan_status','indexedLabel','prediction')\
.filter(result.loan_status != 'Fully Paid').\
filter(result.indexedLabel == result.prediction).count()

18007

In [108]:
result.select('loan_status','indexedLabel','prediction')\
.filter(result.loan_status != 'Fully Paid').\
filter(result.indexedLabel != result.prediction).count()

12259

In [109]:
result.select('loan_status','indexedLabel','prediction','prediction_label')\
.filter(result.loan_status == 'Fully Paid').show(20)

+-----------+------------+----------+----------------+
|loan_status|indexedLabel|prediction|prediction_label|
+-----------+------------+----------+----------------+
| Fully Paid|         0.0|       0.0|      Fully Paid|
| Fully Paid|         0.0|       0.0|      Fully Paid|
| Fully Paid|         0.0|       1.0|     Charged Off|
| Fully Paid|         0.0|       1.0|     Charged Off|
| Fully Paid|         0.0|       1.0|     Charged Off|
| Fully Paid|         0.0|       0.0|      Fully Paid|
| Fully Paid|         0.0|       0.0|      Fully Paid|
| Fully Paid|         0.0|       1.0|     Charged Off|
| Fully Paid|         0.0|       0.0|      Fully Paid|
| Fully Paid|         0.0|       0.0|      Fully Paid|
| Fully Paid|         0.0|       0.0|      Fully Paid|
| Fully Paid|         0.0|       0.0|      Fully Paid|
| Fully Paid|         0.0|       0.0|      Fully Paid|
| Fully Paid|         0.0|       0.0|      Fully Paid|
| Fully Paid|         0.0|       0.0|      Fully Paid|
| Fully Pa

In [110]:
result.select('loan_status','indexedLabel','prediction','prediction_label')\
.filter(result.loan_status != 'Fully Paid').show(20)

+-----------+------------+----------+----------------+
|loan_status|indexedLabel|prediction|prediction_label|
+-----------+------------+----------+----------------+
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged Off|         1.0|       0.0|      Fully Paid|
|Charged Off|         1.0|       0.0|      Fully Paid|
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged Off|         1.0|       0.0|      Fully Paid|
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged Off|         1.0|       0.0|      Fully Paid|
|Charged Off|         1.0|       0.0|      Fully Paid|
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged Off|         1.0|       0.0|      Fully Paid|
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged Off|         1.0|       1.0|     Charged Off|
|Charged O

In [111]:
result.select('loan_status','indexedLabel','prediction')\
.filter(result.indexedLabel != result.prediction).\
filter(result.loan_status == 'Fully Paid').count()

12723

In [112]:
result.select('loan_status','indexedLabel','prediction')\
.filter(result.indexedLabel != result.prediction)\
.filter(result.loan_status == 'Charged Off').count()

12259

## Calculation for Confusion Matrix.

In [113]:
def eval_metrics(lap):

    tp = float(len(lap[(lap['loan_status']=='Charged Off') &\
                       (lap['prediction_label']=='Charged Off')]))
    
    fn = float(len(lap[(lap['loan_status']=='Charged Off') &\
                       (lap['prediction_label']=='Fully Paid')]))

    tn = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                       (lap['prediction_label']=='Fully Paid')]))

    fp = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                       (lap['prediction_label']=='Charged Off')]))


    try:
        positivepredictivevalue = tp / (tp+fp)
        negativepredictivevalue = tn / (tn+fn)
        sensitivity = tp / (tp+fn)
        specificity = tn / (tn+fp)
        accuracy = (tp+tn) / (tp+tn+fp+fn)
        return {'PPV': positivepredictivevalue, 'Sensitivity': sensitivity,\
            'NPV':negativepredictivevalue, 'Specificity': specificity, 'Accuracy': accuracy}
    except ZeroDivisionError:
        return 0

    

In [114]:
! pip install pandas

You are using pip version 9.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [115]:
lap = result.select("loan_status", "prediction_label").toPandas()

In [116]:
#lap

In [117]:
m = eval_metrics(lap)

In [118]:
m

{'Accuracy': 0.6159038145169969,
 'NPV': 0.6427093351986244,
 'PPV': 0.5859746176374878,
 'Sensitivity': 0.5949580387233199,
 'Specificity': 0.6341337167505392}

In [119]:
tp = float(len(lap[(lap['loan_status']=='Charged Off') &\
                   (lap['prediction_label']=='Charged Off')])) #positive = Charged Off

In [120]:
tp

18007.0

In [121]:
tn = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                   (lap['prediction_label']=='Fully Paid')])) #negative = Fully Paid

In [122]:
tn

22052.0

In [123]:
fp = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                   (lap['prediction_label']=='Charged Off')]))

In [124]:
fn = float(len(lap[(lap['loan_status']=='Charged Off') &\
                   (lap['prediction_label']=='Fully Paid')]))

In [125]:
#! pip install plotly

In [126]:
#! pip install chart-studio

In [127]:
#model1.save('./model_24Nov2019/')

In [128]:
#model1

In [129]:
#! ls -l ./model_24Nov2019/stages/

In [ ]:
#m

# Jan2020

In [130]:
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import Bucketizer

In [131]:
def get_model(df,categoricalCols,continuousCols,\
              discretedCols,split_range,labelCol):

    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
    from pyspark.sql.functions import col
    
    
    labelIndexer = StringIndexer(inputCol=labelCol,\
                             outputCol='indexedLabel',\
                             handleInvalid='keep')

    indexers = [ StringIndexer(handleInvalid='keep',\
                               inputCol=c, outputCol="{0}_indexed".format(c))
                 for c in categoricalCols ]

    # default setting: dropLast=True
    encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(),
                 outputCol="{0}_encoded".format(indexer.getOutputCol()))
                 for indexer in indexers ]
    discretizers = [ Bucketizer(inputCol=d, outputCol="{0}_discretized".format(d)\
                 ,splits=split_range)
                 for d in discretedCols ]
    
    
    featureCols = ['features']
    assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders]
                                + continuousCols +\
                                [discretizer.getOutputCol() for discretizer in discretizers], \
                                outputCol='features')
    

    pipeline_feature = Pipeline(stages=[labelIndexer] + indexers + encoders + discretizers + \
                        [assembler])
    
    
    
    train_df_features = pipeline_feature.fit(training).transform(training)
    
    layers = [
    train_df_features.schema["features"].metadata["ml_attr"]["num_attrs"],
    20, 10, 5, 2]
    
    
    clf = MultilayerPerceptronClassifier(labelCol='indexedLabel',featuresCol='features'\
                                     ,layers=layers)
    
    
    pipeline = Pipeline(stages=[labelIndexer] + indexers + encoders + discretizers + \
                        [assembler] + [clf])
    


    model=pipeline.fit(df)


    return model

In [132]:
training

DataFrame[loan_amnt: double, term: double, int_rate: double, installment: double, grade: string, emp_length: string, home_ownership: string, annual_inc: double, verification_status: string, loan_status: string, purpose: string, addr_state: string, dti: double, delinq_2yrs: string, earliest_cr_line: string, inq_last_6mths: string, open_acc: double, pub_rec: double, revol_bal: double, revol_util: double, total_acc: double, last_credit_pull_d: string]

In [133]:
training.select('grade').describe().show()

+-------+------+
|summary| grade|
+-------+------+
|  count|261081|
|   mean|  null|
| stddev|  null|
|    min|     A|
|    max|     G|
+-------+------+



In [134]:
catcols = [\
           'emp_length',\
           'home_ownership',\
           'verification_status',\
           'purpose',\
           'grade',\
           'addr_state'\
          ]

num_cols = [\
            #'dti',\
            'loan_amnt',\
            'int_rate',\
            'installment',\
            'annual_inc',\
            #'revol_bal',\
            #'delinq_2yrs',\
            'pub_rec',\
            #'revol_util'\
            'total_acc'\
           ]

discols = [\
           #'pub_rec',\
           'dti',\
           #'installment'\
          ]



labelCol = 'loan_status'

splits = [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, float("inf")]

In [135]:
from itertools import combinations 
list(combinations(catcols, 2))

[('emp_length', 'home_ownership'),
 ('emp_length', 'verification_status'),
 ('emp_length', 'purpose'),
 ('emp_length', 'grade'),
 ('emp_length', 'addr_state'),
 ('home_ownership', 'verification_status'),
 ('home_ownership', 'purpose'),
 ('home_ownership', 'grade'),
 ('home_ownership', 'addr_state'),
 ('verification_status', 'purpose'),
 ('verification_status', 'grade'),
 ('verification_status', 'addr_state'),
 ('purpose', 'grade'),
 ('purpose', 'addr_state'),
 ('grade', 'addr_state')]

In [ ]:
for i in range(len(catcols)):
    for j in (list(combinations(catcols, i+1))):
        ml_model = get_model(training,j\
                             ,num_cols,discols, splits, labelCol)
        lap2 = ml_model.transform(test)\
        .withColumn('prediction_label',spark_labeling(col('prediction')))\
        .toPandas()
        m2 = eval_metrics(lap2)
        print("Categorical Features: {0} ได้ Model ที่มี {1}"\
              .format(j,m2),"\n")
        

Categorical Features: ('emp_length',) ได้ Model ที่มี {'PPV': 0.5862117186407495, 'Sensitivity': 0.6925262670983943, 'NPV': 0.6822372464658881, 'Specificity': 0.5745506829618979, 'Accuracy': 0.6294491167109977} 

Categorical Features: ('home_ownership',) ได้ Model ที่มี {'PPV': 0.5970266513567414, 'Sensitivity': 0.6528117359413202, 'NPV': 0.6710802266253483, 'Specificity': 0.6165061107117182, 'Accuracy': 0.6334004704724712} 

Categorical Features: ('verification_status',) ได้ Model ที่มี {'PPV': 0.5960496682695234, 'Sensitivity': 0.6470957510077314, 'NPV': 0.6681167075785353, 'Specificity': 0.6183177570093458, 'Accuracy': 0.6317092295628911} 

Categorical Features: ('purpose',) ได้ Model ที่มี {'PPV': 0.5779758284187978, 'Sensitivity': 0.7220974030264984, 'NPV': 0.6910900543558102, 'Specificity': 0.5411071171818835, 'Accuracy': 0.6253286388585662} 

Categorical Features: ('grade',) ได้ Model ที่มี {'PPV': 0.6002098969596743, 'Sensitivity': 0.6235710037666028, 'NPV': 0.660892341578117, 